# scVI Liver Runtime

In this notebook, we will analyze the scaleability of the scVI method to larger datasets. We fit scVI on various percentages of the Liver dataset (which contains over 100000 cells). We fit scVI on subsets of this data ranging from 10% up to 100% of the full liver data.

In [1]:
"""Broadly useful python packages"""
import pandas as pd
import os
import numpy as np
import pickle
from copy import deepcopy
from shutil import move, rmtree
import warnings
from memory_profiler import memory_usage
from time import time

"""Machine learning and single cell packages"""
import sklearn.metrics as metrics
from sklearn.metrics import adjusted_rand_score as ari, normalized_mutual_info_score as nmi
import scanpy as sc
from anndata import AnnData
import seaborn as sns

"""import scVI Package"""
import scvi
from scvi.model import SCVI

In [2]:
"""Miscellaneous useful functions"""

def read_liver_data(path, cache=True):
    adata = sc.read_mtx(os.path.join(path, 'matrix.mtx')).T
    genes_file = pd.read_csv(os.path.join(path, 'genes.tsv'), sep='\t')
    barcodes_file = pd.read_csv(os.path.join(path, 'barcodes.tsv'), sep='\t')

    adata.var.index = genes_file["genename"]
    adata.obs.index = barcodes_file["cellname"]
    adata.obs = barcodes_file
        
    sc.pp.filter_cells(adata, min_genes = 200)
    mito_genes = adata.var_names.str.startswith('mt-')
    adata.obs['percent_mito'] = np.sum(
        adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1
    adata.obs['n_counts'] = adata.X.sum(axis=1).A1
    adata = adata[adata.obs['percent_mito'] < 0.2, :]
    sc.pp.filter_genes(adata, min_cells = 30)

    return adata

def build_dir(dir_path):
    subdirs = [dir_path]
    substring = dir_path

    while substring != '':
        splt_dir = os.path.split(substring)
        substring = splt_dir[0]
        subdirs.append(substring)
        
    subdirs.pop()
    subdirs = [x for x in subdirs if os.path.basename(x) != '..']

    n = len(subdirs)
    subdirs = [subdirs[n - 1 - x] for x in range(n)]
    
    for dir_ in subdirs:
        if not os.path.isdir(dir_):
            os.mkdir(dir_)

def convert_string_to_encoding(string, vector_key):
    """A function to convert a string to a numeric encoding"""
    return np.argwhere(vector_key == string)[0][0]

def convert_vector_to_encoding(vector, printkey = False):
    """A function to convert a vector of strings to a dense numeric encoding"""
    vector_key = np.unique(vector)
    if printkey:
        print(pd.Series(vector_key))
    
    vector_strings = list(vector)
    vector_num = [convert_string_to_encoding(string, vector_key) for string in vector_strings]
    
    return vector_num
            
def run_scvi(adata):
    batch_vec = adata.obs['sampleid'].values
    scvi.data.setup_anndata(adata, batch_key="sampleid")

    vae = SCVI(adata, n_layers = 2)

    ELBO_best = np.inf
    patience = 0
    ES_stop = 9
    n_epoch = 0

    while patience < ES_stop and n_epoch < 200:
        vae.train(n_epochs = 1)
        ELBO = - vae.get_elbo()

        if np.isnan(ELBO):
            ELBO = np.inf

        if ELBO + 10**(-3) < ELBO_best:
            patience = 0
            ELBO_best = ELBO
        else:
            patience += 1

        n_epoch += 1
            
    latent = vae.get_latent_representation()
    
    normalized_values = vae.get_normalized_expression(
        library_size=1e4, 
        transform_batch = "F2", 
        n_samples=10, 
        return_mean=True,
    )
        
def profile(frac):
    np.random.seed(11111)
    indices = np.random.choice(range(adata.shape[0]), size = round(frac * adata.shape[0]), replace = False)
    tmp = adata[indices].copy()
    
    sc.pp.filter_genes(tmp, min_cells = 1)
    start = time()
    run = memory_usage((run_scvi, (), {'adata': tmp}))
    final = time() - start
    peak_memory = max(run) - min(run)
    stats_zscore = final, peak_memory, "scVI", int(100*frac)
    
    return stats_zscore

## Figure Data

In [3]:
build_dir("../Figures/liver")
profile_stats = {"Time (Seconds)": [] , "Memory (MiB)": [], "Method": [], 'Percent': []}
profile_stats = pd.DataFrame(profile_stats)

## Read in Data

In [4]:
adata = read_liver_data("../Data/liver", cache = True)

/Users/jlakkis/anaconda3/envs/cardec_alternatives/lib/python3.7/site-packages/anndata/_core/anndata.py:119: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
Trying to set attribute `.var` of view, copying.


## Profile Memory and Speed

In [ ]:
fracs = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0] #kernel may crash after frac = 0.6, depending on hardware

n = 0
for frac in fracs:
    profile_stats.loc[n] = profile(frac)
    profile_stats.to_csv("../Figures/liver/scVI_profile.csv")
    n = n + 1